In [ ]:
# --- COLAB SETUP START ---
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    # 1. Clone the repository
    !git clone https://github.com/MasumRab/gemini-fullstack-langgraph-quickstart
    %cd gemini-fullstack-langgraph-quickstart/backend

    # 2. Prepare Environment (Resolving Conflicts)
    import sys
    print("Uninstalling conflicting pre-installed packages...")
    !pip uninstall -y google-ai-generativelanguage google-generativeai tensorflow grpcio-status

    # Pre-install PyTorch Nightly if Python 3.13+ is detected
    if sys.version_info >= (3, 13):
        print("Detected Python 3.13+. Installing PyTorch Nightly...")
        !pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

    # 3. Install Backend
    !pip install .

    # 4. Set API Key
    import os
    from google.colab import userdata
    try:
        os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
    except:
        print("Please enter your Gemini API Key:")
        os.environ["GEMINI_API_KEY"] = input()
# --- COLAB SETUP END ---

# MCP Tools: Planned Tool Use

This notebook demonstrates the Model Context Protocol (MCP) integration, specifically:
1. Setting up a Filesystem MCP Server
2. Planning tool sequences with an LLM
3. Executing file operations safely

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import shutil

# Add backend/src to path
sys.path.append(os.path.abspath("../backend/src"))

from agent.mcp_server import FilesystemMCPServer
from agent.mcp_client import MCPToolUser
from langchain_google_genai import ChatGoogleGenerativeAI

## 1. Setup MCP Server

We create a sandbox directory and initialize the server allowing access only to that directory.

In [ ]:
# Setup sandbox
sandbox_dir = "./mcp_sandbox"
os.makedirs(sandbox_dir, exist_ok=True)

# Initialize Server
fs_server = FilesystemMCPServer(allowed_paths=[sandbox_dir])
print(f"Filesystem Server initialized. Root: {sandbox_dir}")

## 2. Initialize Tool User (Client)

The client aggregates tools from servers and handles planning.

In [ ]:
mcp_client = MCPToolUser(mcp_servers=[fs_server])

# List available tools
print("Available Tools:")
for name in mcp_client.tool_registry:
    print(f"- {name}")

## 3. Basic Tool Operations (Manual Test)

In [ ]:
import asyncio

async def run_manual_test():
    # Write
    print("Writing file...")
    res = await mcp_client.execute_tool(
        "filesystem.write_file", 
        path=f"{sandbox_dir}/test.txt", 
        content="Hello MCP World!"
    )
    print("Write Result:", res)

    # Read
    print("\nReading file...")
    res = await mcp_client.execute_tool(
        "filesystem.read_file", 
        path=f"{sandbox_dir}/test.txt"
    )
    print("Read Result:", res)

await run_manual_test()

## 4. Tool Planning Demo

We ask the LLM to plan a sequence of operations.

In [ ]:
# Initialize LLM
try:
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
except Exception as e:
    print("Using Mock LLM")
    class MockLLM:
        def invoke(self, prompt): return '```json\n[{"tool": "filesystem.write_file", "params": {"path": "./mcp_sandbox/plan.txt", "content": "Step 1: Done"}}]\n```'
    llm = MockLLM()

# Scenario
task = f"Create a new directory named 'reports' inside '{sandbox_dir}' and write a file named 'summary.md' inside it with the text 'Analysis Complete'."

# 1. Plan
plan = mcp_client.plan_tool_sequence(task, llm)
print("Generated Plan:")
print(json.dumps(plan, indent=2))

# 2. Execute
print("\nExecuting Plan...")
import json
if plan:
    results = await mcp_client.execute_plan(plan)
    for r in results:
        print(f"Tool: {r['tool']}, Success: {r['result'].get('success')}")
else:
    print("Plan generation failed or empty.")

## 5. Security & Constraints

Verify that we cannot write outside the sandbox.

In [ ]:
async def test_security():
    print("Attempting to write to /tmp (outside sandbox)...")
    res = await mcp_client.execute_tool(
        "filesystem.write_file", 
        path="/tmp/hacked.txt", 
        content="Should fail"
    )
    print("Result:", res)

await test_security()